In [1]:
%autosave 60

Autosaving every 60 seconds


In [2]:
import string
string.ascii_letters + string.digits
string.printable

'0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0c'

In [3]:
# class for the dataset:
import torch
# class for text-datasets
# Jama, NEJM and lancet
from torch.utils.data import Dataset, DataLoader
import unicodedata
class TextDataset(Dataset):

    SPECIAL_CHARS = [None, "\n"]
    def __len__(self):
        return len(self.total_examples)
        pass
 
    # generates tensors for the vocab
    # i.e. assigns a number to each character
    # Q: possible to do one-pass?
    def make_vocab(self):
        
        self.char2index = {}
        self.index2char = {}
        index = 0
        with open(self.text_file) as file:
            for line in file:
                for char in line:
                    if char not in self.char2index:
                        self.char2index[char] = index
                        self.index2char[index]  = char
                        index+=1
        
        for char in self.SPECIAL_CHARS:
#             print("this is the char")
            if char not in self.char2index:
                self.char2index[char] = index
                self.index2char[index] = char
                index += 1
        
        print(self.char2index[None])
        # potentially need to add 2 for the None and \n characters
        self.vocab_size = len(self.char2index)
    
    # this allows the model to handle all possible strings passed into it!!
    def full_vocab(self):
        self.char2index = {}
        self.index2char = {}
        index = 0
        all_letters = string.printable 
        for char in all_letters:
            if char not in self.char2index:
                self.char2index[char] = index
                self.index2char[index]  = char
                index+=1
                
        for char in self.SPECIAL_CHARS:
#             print("this is the char")
            if char not in self.char2index:
                self.char2index[char] = index
                self.index2char[index] = char
                index += 1
                
        self.vocab_size = len(self.char2index)   
        
        
    
    
    def generate_tensor_for_char(self, char):
        temp = torch.zeros(1, self.vocab_size)
        temp[0][self.char2index[char]] = 1 
        return temp

#     def generate_char_from_tensor(self, char):
#         temp = torch.zeros(self.vocab_size, 1)
#         temp[self.char2index[char]][0] = 1 
#         return temp
    def lineToTensor(self,line):
        tensor = torch.zeros(len(line), 1, self.vocab_size)
#         print(line)
        for li, letter in enumerate(line):
            tensor[li][0][self.char2index[letter]] = 1
        return tensor
    
    # wrap the lines!
    def train_example_builder(self, line):
        tensor = torch.zeros(len(line), 1, self.vocab_size)
        for li, letter in enumerate(line):
            tensor[li][0][self.char2index[letter]] = 1
        return tensor
      
#     could do more of a common build api
    def get_None_tensor(self):
        tensor = torch.zeros(1, 1, self.vocab_size)
        tensor[0][0][self.char2index[None]] = 1
        return tensor
    
    def list_to_tensor(self, input_list):
        tensor = torch.zeros(len(input_list), 1, self.vocab_size)
        for elt in input_list:
            tensor[li][0][self.char2index[letter]] = 1
            
#     def get_training_pair(self, line):
        
        
    
    def get_new_line_tensor(self):
        tensor = torch.zeros(1, 1, self.vocab_size)
        tensor[0][0][self.char2index["\n"]] = 1
        return tensor

        
#         tensor[li][0][self.char2index[letter]] 
#         temp 
    def unicodeToAscii(self,s):
        return ''.join(
            c for c in unicodedata.normalize('NFD', s)
            if unicodedata.category(c) != 'Mn'
            and c in self.char2index
        )

#     OK, so given the raw lines of text, then we want to get it! (but it is json, so we need to process it anyways)
# 
    def get_test_split():
        return self.test_split
        pass
    def __init__(self, text_files,  convert_to_ascii = True):
        self.string = string
        self.text_files = text_files

        self.training_examples = []

        total_inputs = []
        total_outputs = []
        
#         make the vocab
        self.full_vocab()
    
        from tqdm import tqdm
        for target_class, file_name in enumerate(tqdm(text_files)):
            with open(file_name,  encoding="utf8") as file:
                for counter, raw_line in enumerate(file):
#                     if counter > 500:
#                         break

                    line = self.unicodeToAscii(raw_line) if convert_to_ascii else raw_line
                    line = line[0:139]
                    # zero pad to start


    #                 think about how to make this a vector or tensor line! 
    #                 print()

    #                 inputs = self.get_None_tensor() + line_to_tensor(line)
    #                 inputs = [None] + [x for x in line]
                    inputs = self.lineToTensor([None] + [x for x in line])

                    targets = torch.zeros((1,4))  # we need a 0 as well!
                    targets[0,target_class] = 1
                
#                     print(targets)

                    # append: will be a list of list (of lines)
                    # extend: will be a list of lines
                    total_inputs.append(inputs)
                    total_outputs.append(targets)

        assert len(total_inputs) == len(total_outputs)
        
        import random
        
        
        # this is now a LIST of lists!
        self.total_examples = list(zip(total_inputs, total_outputs))
        random.shuffle(self.total_examples)
        train_split = self.total_examples[0:32000]
        test_split = self.total_examples[32000:-1] #all the rest are test exampels
        print(len(train_split))
        print(len(test_split))
        self.test_split = test_split
        self.total_examples = train_split
        


        #                 for char in line:
        # how many chars before the target? or all chars before the target?
        # so we simply pair up x with [x+1] i think
        # ok, and then we iterate them in the loader?
        #                     inputs = [x for x in ]
        #                     targets = []

        # we should also have some big list of entries 

        pass
    
    
    # ok, now let us just try adding the transforms
    # actually the transforms should take responsibility for all of the vocab stuff
    
    # number of items in the dataset. This involves a rather large computation!
    # we can either do a feed forward cllm in the style of bengio, or we can just 
    # do the RNN approach (as specified by Sean Robertson in his tutorial)

    # the indices should represent examples inside the list
    # one strange thing is the following: we need to precompute al the indices earlier, imo, and then simply return the list
    # at that element! (perhaps with some transforms!)
    def __getitem__(self, index):
        return self.total_examples[index]

        # build a list, return elements from that list? we need to internally keep track of some index
        pass



# ok, so it works now!

In [4]:
files = [r"reddit_data/Left-reduced",r"reddit_data/Center-reduced", 
        r"reddit_data/Right-reduced", r"reddit_data/Alt-reduced"]
abc = TextDataset( files, True)


#think about unicode issues


100%|██████████| 4/4 [00:29<00:00,  7.41s/it]

32000
8028


In [5]:
print(abc.char2index[None])

100


In [6]:
abc.generate_tensor_for_char("a")
# abc.index2char[2]
abc.lineToTensor("abcd")
    

tensor([[[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
           0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.]],

        [[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
  

In [7]:


for (a,b) in abc:
    pass
#     this should give two lines/tests
#     print(a,b)
#     print()
    
#     realistically, we should also have it as tensors/vectors
# finally, we should also have some way where we run the rnn forward and 
#  iteratoring over the set, does not consume the elements! It is not a genertor! Rather, we simply index into the elements we would like
for(a,b) in abc:
    pass
#     print("ok")

In [8]:
char_x = [None] + ["e"]

In [9]:
# ok, so now let's just do the stuff
# the RNN actually makes one-character over predictions! 
# rnn_forward is a utility function that does a lot of stuff
len(char_x)


2

In [10]:
# ok, so now we can start building the network that will process these!!
# simply: iterate the dataset, and run train on it, do the log likelihood and etc.

import torch.nn as nn
import torch.nn.functional as F
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        combined = F.relu(combined)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)
    
n_hidden = 256


rnn = RNN(abc.vocab_size, n_hidden, 4) #since 4 categories

In [11]:
abc.lineToTensor("we")

tensor([[[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
           0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.]],

        [[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
  

In [12]:

# WE NEED THIS LINE TO RESET THE RNN!!
rnn = RNN(abc.vocab_size, n_hidden, 4)
learning_rate = 0.0005 # If you set this too high, it might explode. If too low, it might not learn
#  we had to set it a big lower to force convergence

import torch.optim as optim
optimizer = optim.Adam(rnn.parameters())

# returns the loss for a line

def train(input_tensor, target_tensor):
    
        
#     print(input_tensor)
#     print(input_tensor.shape)

#     print(target_tensor)

    optimizer.zero_grad()
    
#     print (input_tensor)
    
#     print("changed3")
#     print(input_tensor[1])
    criterion = nn.NLLLoss()    
    hidden = rnn.initHidden()
    loss = 0
    rnn.zero_grad()
#     print(input_tensor.size())
#     print(target_tensor[0])
#     print(target_tensor)
#     print(input_tensor.size()[0])
    
#     we want EVERYTHING to come and output a single value
    for i in range(input_tensor.size()[0]):
        output, hidden = rnn(input_tensor[i], hidden)
        
        
        
#         find where the one is:
# 
#         torch.tensor([all_categories.index(category)], dtype=torch.long)
    
    true_target_tensor = torch.tensor([torch.argmax(target_tensor)], dtype=torch.long)
    
#     true_target_tensor = target_tensor
#     print(output)
#     print(output.shape)
#     print(true_target_tensor)
#     print(true_target_tensor.shape)
# it needs a single max value interestingly!
#     Evaluate the loss on each character!
    
#     some "gradient clipping"; the loss has no effect in this case

    loss += criterion(output,true_target_tensor)
    loss.backward()
        
#         expects JUST a single target class:

#     print ("loss for i " + str(loss.item()))    
#         print()
# should be do the loss on each thing, or each batch! no, we should accumulate it! 
    loss_val = loss.item()
    if loss_val < 5:
        optimizer.step()
    else:
#         print("clipping")
#         print(loss_val)
        loss_val = 5
    
    # Add parameters' gradients to their values, multiplied by learning rate
#     for p in rnn.parameters():
#         p.data.add_(-learning_rate, p.grad.data)

    return output, loss_val

# why not run this just on all the tensors!! 

# then, we just call it with the dataset class, and then just iterate everything!!

In [13]:
abc.index2char[0]
abc.index2char[1]

abc.char2index["A"]

# ok some weird stuff here...


36

In [14]:
# for (x,y) in abc:
#     print(y)


In [ ]:
# del rnn
# rnn = RNN(abc.vocab_size, n_hidden, abc.vocab_size)

epoch_loss = 0
# loss every k iters
# 
total_loss = 0
total_length  = 0
epoch_length = 0
other_loss = 0

from tqdm import tqdm
num_epochs = 10
for k in range(num_epochs ):
    
    for i,(x,y) in enumerate(tqdm(abc)):
    #     print(x)
    #     print(x)
    #     print(y)
        _, loss = train(x,y)
        epoch_loss += loss
        epoch_length += x.size()[0]
        total_loss += loss
        total_length += x.size()[0]
        other_loss +=loss
#         print(loss)
    #     print(i)
    #     abc = i
        if i % 500 == 0:
            print ("loss for {} is {}".format(i,other_loss/500))
            other_loss = 0
#         break

    
    print("epoch {} loss is {}".format(k, epoch_loss/i))
    print("per character loss is {}".format(epoch_loss/epoch_length))
    epoch_loss = 0
    epoch_length = 0
#     ideally: we would be able to compute the quintile losses
    

  0%|          | 2/32000 [00:00<58:55,  9.05it/s]

loss for 0 is 0.0026408445835113523


  2%|▏         | 505/32000 [00:21<22:40, 23.15it/s]

loss for 500 is 1.3846401236057282


  3%|▎         | 1004/32000 [00:43<22:20, 23.12it/s]

...

 47%|████▋     | 15009/32000 [12:44<14:25, 19.62it/s]

loss for 15000 is 1.346876716852188


 48%|████▊     | 15505/32000 [12:58<13:47, 19.92it/s]

loss for 15500 is 1.3709960869550706


 49%|████▉     | 15758/32000 [13:10<13:34, 19.94it/s]

In [ ]:
# Saves the entire model under the given filename
def save_model(model,filename):
    torch.save(model, filename)
    
    return

# Saves the model parameters only (for maximum usability)

def save_model_params(model, filename):

    torch.save(model.state_dict(), filename)

    return filename

#  ideally, I could get per character loss, then I would also be able to get the total expected loss
# so: we want to get total loss, then divide it by all the length of the number of examples we saw

In [ ]:
def load_model_from_params(filename):
    model = RNN(abc.vocab_size, n_hidden, abc.vocab_size)
    model.load_state_dict(torch.load(filename))
    return model #vs return model.eval()??

In [ ]:
filename = "reddit_draft_new.pt"

if True:

    filename = save_model_params(rnn, "reddit_draft_new.pt")

In [ ]:
if True:
    eval_model = load_model_from_params(filename)
    eval_model.eval() # this is a mutating operation


In [ ]:

# returns the loss for the entire line (requires normalization for comparison)
def get_loss_on_line(rnn, line):
    input_tensor =  abc.lineToTensor([None] + [x for x in line])
    target_tensor = abc.lineToTensor([x for x in line] + ["\n"])
    criterion = nn.NLLLoss()    
    hidden = rnn.initHidden()
    loss = 0
    rnn.zero_grad()
    
    
#     print(input_tensor.size())
#     print(target_tensor[0])
#     print(target_tensor)
    for i in range(input_tensor.size()[0]):
        output, hidden = rnn(input_tensor[i], hidden)
        
        
        # had to do this simply because of how the NLLL and CrossEntropy are defined
        max_tensor = torch.argmax(target_tensor[i])
        ind = (torch.argmax(target_tensor[i])).item()
        true_target_tensor = torch.tensor([ind], dtype=torch.long)
    
#     Evaluate the loss on each character!
        loss += criterion(output, true_target_tensor)
    
    return loss.item()



In [ ]:
# out = eval_model)

# so there's a couple possibilities: we can have jim and ask to predict as well as just get the loss of the example
# out = train()
# ()
#lineToTensor

# get it as tensors, then run it, then do loss addition

STRING_TO_TEST =  "David"
print(get_loss_on_line(eval_model, STRING_TO_TEST )/len(STRING_TO_TEST ))

# we now want to implement sampling:
#  is that useful though? 
# will it be good?
# instead, we can also try and get text for the inputs 
#  then train on that, then run it on some numerical numbers

# visualization would be super useful 
# for pretty printing, we can colour the terminal output
# otherwise, we will need to somehow encode the colour so that it is output to the file
# 


In [ ]:
#  print("\x1b[31m\"red\"\x1b[0m")

# if no thresholding, then we should at least like somehow graphically display?
# no,... this is a different concern. Don't put frontend and backend together! (or visualization and etc.)


In [ ]:
from colorama import Fore

In [ ]:
# print(Fore.RED + "yoo")

# print(Fore.BLUE + "haha")

# if on average more than twice as high loss on this sample, then we should flag it for further review
THRESHOLD =  1.8332151545300774 * 2


# some more powerful model: uses LSTM as well as additional fields and operators!
# ok, so now let us just iterate the text file
with open("data\\test-cases.txt", "r") as file:
    for line in file:
        example_loss = get_loss_on_line(eval_model, line)/len(line )
        if (example_loss > THRESHOLD):
            print(Fore.RED + line)
        else:
            print(Fore.BLUE + line)
        
        print("loss" + str(example_loss))
    
# perhaps we should train on google news corpus for a little while..
# this 
    

In [ ]:
import nltk

In [ ]:
from nltk.corpus import reuters

In [ ]:
print("ok")

sentences = reuters.raw().splitlines()
# print(reuters.raw())


In [ ]:
print(type(sentences))
print(len(sentences))

In [ ]:

# these are not exactly tokenized on sentences; but that is OK!
sentences[:100]

with open("reuters_news.txt", "w") as file:
    file.write(reuters.raw())

In [ ]:
# we need a smaller dataset for development
# so, let's cut up the reuters into smaller (for example, load just the first K bytes of the file.)
import time

def make_proto_dataset(size):
    with open("reuters_news_{}.txt".format(size), "w") as file:
        
        
        file.write('\n'.join(reuters.raw().splitlines()[0:size]))
    print("file created with {} lines".format(size))
        


In [ ]:
#     the question now is how many neurons and the architecture to use
# we can also look into enhancement with the tqdm inside the object?
start = time.time()


make_proto_dataset(10000)
elapsed = time.time() - start
print("elapsed time is {}".format(elapsed ))


In [ ]:
# reuters dataset has umlauts and other non-standard characters, so we should try and account for that! 
# also: we should try and do a sort of full merge: merge between the train dataset and test dataset vocabs
# other than that, we can just apply a regularizer to both sets, to get out the processed data
# for now, let's just apply a regularization

start = time.time()
reuters_dataset = TextDataset( "reuters_news_10000.txt", True)
elapsed = time.time() - start
print("elapsed time is {}".format(elapsed ))

# jeez, so it takes more than 8 minutes to create the dataset!

In [ ]:
print(len(reuters_dataset))
from tqdm import tqdm
def general_train_function(dataset):
# del rnn
# rnn = RNN(abc.vocab_size, n_hidden, abc.vocab_size)

    epoch_loss = 0
    # loss every k iters
    # 
    loss_per_k = 0
    
    total_loss = 0
    total_length  = 0
    epoch_length = 0


    num_epochs = 10
    
    for k in range(num_epochs ):

        for i,(x,y) in enumerate(tqdm(dataset)):
        #     print(x)
        #     print(x)
        #     print(y)
            _, loss = train(x,y)
            epoch_loss += loss
            epoch_length += x.size()[0]
            total_loss += loss
            total_length += x.size()[0]
            loss_per_k += loss
        #     print(i)
        #     abc = i
        
            # for the first 100 iters, print the loss of every line!
#             if i < 100:
#                 print ("loss for {} is {}".format(i,epoch_loss/(i+1)))
                
            
            if i % 100 == 0 and i != 0:
                print ("loss for {} is {}".format(i,loss_per_k/100))
                loss_per_k = 0
                

        print("epoch {} loss is {}".format(k, epoch_loss/i))
        print("per character loss is {}".format(epoch_loss/epoch_length))
        epoch_loss = 0
        epoch_length = 0
#     ideally: we would be able to compute the quintile losses
    

In [ ]:

# WE NEED THIS LINE TO RESET THE RNN!!
rnn = RNN(reuters_dataset.vocab_size, 512, reuters_dataset.vocab_size)
learning_rate = 0.0005 # If you set this too high, it might explode. If too low, it might not learn
#  we had to set it a big lower to force convergence

import torch.optim as optim
optimizer = optim.SGD(rnn.parameters(), lr = 0.0001)

# returns the loss for a line

def general_train(input_tensor, target_tensor):
    
        
    
    optimizer.zero_grad()
    
#     print (input_tensor)
    
#     print("changed3")
#     print(input_tensor[1])
    criterion = nn.NLLLoss()    
    hidden = rnn.initHidden()
    loss = 0
    rnn.zero_grad()
#     print(input_tensor.size())
#     print(target_tensor[0])
#     print(target_tensor)
    for i in range(input_tensor.size()[0]):
        output, hidden = rnn(input_tensor[i], hidden)
        
        
        
#         find where the one is:
# 
#         torch.tensor([all_categories.index(category)], dtype=torch.long)
        max_tensor = torch.argmax(target_tensor[i])
#         print(max_tensor)
        ind = (torch.argmax(target_tensor[i])).item()
#         print("this the target")
#         print(target_tensor[i])
#         print(ind)
        true_target_tensor = torch.tensor([ind], dtype=torch.long)
    
#     Evaluate the loss on each character!
        loss += criterion(output, true_target_tensor)
        
#         expects JUST a single target class:

#     print ("loss for i " + str(loss.item()))    
#         print()
# should be do the loss on each thing, or each batch! no, we should accumulate it! 

    loss.backward()
    optimizer.step()
    # Add parameters' gradients to their values, multiplied by learning rate
#     for p in rnn.parameters():
#         p.data.add_(-learning_rate, p.grad.data)

    return output, loss.item()

# why not run this just on all the tensors!! 

# then, we just call it with the dataset class, and then just iterate everything!!

In [ ]:
general_train_function(reuters_dataset)
# perhaps: examine the gradient instead!



In [ ]:

# returns the loss for a line, evaluating on a BASIC rnn, and using the provided optimizer
def prototype_general_train(input_tensor, target_tensor, basic_rnn, optimizer):
    
        
    
    optimizer.zero_grad()
    
    criterion = nn.NLLLoss()    
    hidden = rnn.initHidden()
    loss = 0
    basic_rnn.zero_grad()
    for i in range(input_tensor.size()[0]):
        output, hidden = basic_rnn(input_tensor[i], hidden)
        
        max_tensor = torch.argmax(target_tensor[i])
        ind = (torch.argmax(target_tensor[i])).item()
        true_target_tensor = torch.tensor([ind], dtype=torch.long)
        #     Evaluate the loss on each character!
        loss += criterion(output, true_target_tensor)
        
    loss.backward()
    optimizer.step()

    return output, loss.item()
